# "Conv1d"

In [1]:
import keras
from keras import models, layers
import numpy as np
import matplotlib.pyplot as plt
from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D
import sys
sys.path.append('../')
from utils import DataFeed
from keras import regularizers

Using TensorFlow backend.


### Load Data

In [2]:
data_path='../preprocessing/preprocessed_data'

In [3]:
train_data, train_labels = DataFeed.Dataset.create(data_path, ['train/voxforge', 'train/youtube'], num=50000, use_premade=True)
val_data, val_labels = DataFeed.Dataset.create(data_path, ['val/youtube', 'val/voxforge', 'test/librivox'], num=-1, shuffle=True)

In [4]:
training_generator = DataFeed.DataGenerator(data_path, ['train/voxforge', 'train/youtube', 'train/librivox'], num=-1, batch_size=64, shuffle=True)

In [4]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=5),
             keras.callbacks.ModelCheckpoint('conv1d.h5', monitor='val_loss', save_best_only=True)]

### Define Model

In [79]:
model = models.Sequential()

model.add(layers.Reshape((80000, 1), input_shape=(1, 80000)))

model.add(layers.Conv1D(16, 6, strides=2, activation='relu', padding='same'))
model.add(layers.MaxPooling1D((4)))
model.add(layers.BatchNormalization())

model.add(layers.Conv1D(32, 6, strides=2, activation='relu', padding='same'))
model.add(layers.MaxPooling1D((4)))
model.add(layers.BatchNormalization())

model.add(layers.Conv1D(32, 6, activation='relu', padding='same'))
model.add(layers.MaxPooling1D((4)))
model.add(layers.BatchNormalization())

model.add(layers.Conv1D(64, 6, activation='relu', padding='same'))
model.add(layers.MaxPooling1D((4)))
model.add(layers.BatchNormalization())

model.add(layers.Conv1D(64, 6, activation='relu', padding='same'))
model.add(layers.MaxPooling1D((4)))
model.add(layers.BatchNormalization())

model.add(layers.Conv1D(64, 6, activation='relu', padding='same'))
model.add(layers.MaxPooling1D((4)))
model.add(layers.BatchNormalization())

model.add(layers.Flatten())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

In [80]:
model.compile(optimizer='Rmsprop',
              metrics=['accuracy'],
              loss='categorical_crossentropy')

### Train Model

In [81]:
history = model.fit(x=train_data,
                        y=train_labels,
                        batch_size=32, 
                        epochs=13,
                        verbose=1,
                        validation_data=(val_data, val_labels), 
                        shuffle=True,
                        callbacks=callbacks)

Train on 50000 samples, validate on 11080 samples
Epoch 1/13
50000/50000 [==============================] - 63s 1ms/step - loss: 0.7010 - acc: 0.7000 - val_loss: 1.8989 - val_acc: 0.4961
Epoch 2/13
50000/50000 [==============================] - 59s 1ms/step - loss: 0.4232 - acc: 0.8324 - val_loss: 0.8964 - val_acc: 0.6613
Epoch 3/13
50000/50000 [==============================] - 59s 1ms/step - loss: 0.3395 - acc: 0.8687 - val_loss: 1.1722 - val_acc: 0.6835
Epoch 4/13
50000/50000 [==============================] - 59s 1ms/step - loss: 0.2879 - acc: 0.8891 - val_loss: 0.7409 - val_acc: 0.6881
Epoch 5/13
50000/50000 [==============================] - 59s 1ms/step - loss: 0.2564 - acc: 0.9022 - val_loss: 0.9676 - val_acc: 0.7282
Epoch 6/13
50000/50000 [==============================] - 58s 1ms/step - loss: 0.2381 - acc: 0.9102 - val_loss: 0.7037 - val_acc: 0.7125
Epoch 7/13
50000/50000 [==============================] - 59s 1ms/step - loss: 0.2190 - acc: 0.9183 - val_loss: 1.3154 - val_acc

In [3]:
best_model = keras.models.load_model('conv1d.h5', custom_objects={'Melspectrogram':Melspectrogram, 
                                                                         'Normalization2D': Normalization2D})
test_data, test_labels = DataFeed.Dataset.create(data_path, ['test/voxforge', 'test/youtube'], num=-1, shuffle=True)
best_model.evaluate(x=test_data,
                    y=test_labels), best_model.metrics_names

10000/10000 [==============================] - 8s 753us/step


([0.3471113031387329, 0.8693], ['loss', 'acc'])